In [7]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d
%matplotlib widget
import datetime

In [8]:
traindata = pd.read_csv("train.csv")
testdata = pd.read_csv("test.csv")

In [9]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
traindata.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [11]:
trainLabels = traindata.pop("SalePrice")

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [49]:
for col in traindata.columns:
    if traindata[col].dtype =="object":
        traindata[col]=traindata[col].fillna("None")
        testdata[col] = testdata[col].fillna("None")
    if traindata[col].dtype in ["float64","int64"] :
        traindata[col]=traindata[col].fillna(0.0)
        testdata[col]=testdata[col].fillna(0.0)

In [15]:
traindata = traindata.fillna(0.0)

In [14]:
for col in [col for col in traindata.columns if traindata[col].dtype=="object"]:
    tokenizer = Tokenizer(num_words=len(pd.unique(traindata[col])),lower=False)
    tokenizer.fit_on_texts(traindata[col])
    traindata[col] = pd.DataFrame(tokenizer.texts_to_sequences(traindata[col]))
    testdata[col] = pd.DataFrame(tokenizer.texts_to_sequences(testdata[col]))


In [16]:
traindata.pop("Id")

0          1
1          2
2          3
3          4
4          5
        ... 
1455    1456
1456    1457
1457    1458
1458    1459
1459    1460
Name: Id, Length: 1460, dtype: int64

In [452]:
selecttrain = pd.DataFrame([traindata.LotFrontage,traindata.LotArea,traindata.YrSold]).T

In [271]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(79,name="input"),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(1,name="output")
])

In [272]:
model.compile(loss=tf.keras.losses.mean_squared_logarithmic_error,optimizer='sgd',metrics = [tf.keras.metrics.MeanAbsolutePercentageError()])

In [277]:
model.fit(x=traindata,y=trainLabels,epochs=100,callbacks=[tensorboard_callback])

Epoch 1/100
46/46 [==============================] - 0s 6ms/step - loss: 0.1182 - mean_absolute_percentage_error: 27.6351
Epoch 2/100
46/46 [==============================] - 0s 2ms/step - loss: 0.1180 - mean_absolute_percentage_error: 27.6703
Epoch 3/100
46/46 [==============================] - 0s 2ms/step - loss: 0.1178 - mean_absolute_percentage_error: 27.6200
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 0.1176 - mean_absolute_percentage_error: 27.6199
Epoch 5/100
46/46 [==============================] - 0s 2ms/step - loss: 0.1174 - mean_absolute_percentage_error: 27.4889
Epoch 6/100
46/46 [==============================] - 0s 2ms/step - loss: 0.1172 - mean_absolute_percentage_error: 27.5143
Epoch 7/100
46/46 [==============================] - 0s 4ms/step - loss: 0.1170 - mean_absolute_percentage_error: 27.5053
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 0.1168 - mean_absolute_percentage_error: 27.4761
Epoch 9/100
46/46 [=====

In [276]:
print(model.predict(traindata.head()))
print(trainLabels.head())

[[156152.66]
 [164589.75]
 [180401.6 ]
 [163823.06]
 [212480.92]]
0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


In [ ]:
%tensorboard --logdir logs/fit

In [103]:
model.evaluate(traindata,trainLabels)

46/46 [==============================] - 0s 2ms/step - loss: 0.0121


0.012052532285451889

In [40]:
Id = testdata.pop("Id")

In [104]:
results = pd.DataFrame(model.predict(testdata))

In [105]:
results["Id"] = Id

In [106]:
results.columns = ["SalePrice","Id"]

In [108]:
results

,SalePrice
Id,
1461,125722.453125
1462,249647.546875
1463,171913.265625
1464,191702.500000
1465,186241.453125
...,...
2915,90683.929688
2916,90265.000000
2917,173667.781250


In [107]:
results = results.set_index("Id")

In [76]:
nan_vals = testdata.isna()
nan_cols = nan_vals.any()
colwnan = testdata.columns[nan_cols].to_list()
print(colwnan)

[]


In [109]:
results.to_csv("results.csv")